In [1]:
# We are importing json for reading the json file, Counter to countand also use its builtin functionsl, date time so that we can convert the date of birth and fetch the year of birth and compare it to present date for age and pandas for dataframe
import json
from collections import Counter
from datetime import datetime
import pandas as pd

In [9]:
#In this step we are loading the json file in data
import json
with open('/content/DataEngineeringQ2.json', 'r') as f:
  data = json.load(f)

In [15]:
# In the below step we are assigning the data in patient details and consultation details to the different variables
patient_details = [item['patientDetails'] for item in data]
consultation_data = [item['consultationData'] for item in data]

In [44]:
#5.What is the percentage of female gender after imputation of gender column basis mode.

#In this step we are creating a common gender in which the most repeated gender is assigned
genders = [patient.get('gender') for patient in patient_details if patient.get('gender')]
common_gender = Counter(genders).most_common(1)[0][0]

In [45]:
#In this step we look for the empty values and then change it with the common gender
for patient in patient_details:
    if not patient.get('gender'):
        patient['gender'] = common_gender

In [13]:
# Calculate percentage of females
total_patients = len(patient_details)
female_patients = sum(1 for patient in patient_details if patient['gender'] == 'F')
female_percentage = (female_patients / total_patients) * 100
female_percentage = round(female_percentage, 2)

In [14]:
female_percentage

32.26

In [ ]:
#6. Add ageGroup column and count Adults

In [17]:
#In this step we are assigning a function to calculate the age
def calculate_age(birth_date):
    if not birth_date:
        return None
    birth_date = datetime.strptime(birth_date, '%Y-%m-%dT%H:%M:%S.%fZ')
    today = datetime.now()
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))


In [18]:
# In this step we are adding the age and ageGroup columns
age_groups = {
    'Child': 0,
    'Teen': 0,
    'Adult': 0,
    'Senior': 0
}

In [19]:
#In this step we take the the age from the patient details and thyen pass it through the for loop and count the total number
for patient in patient_details:
    age = calculate_age(patient.get('birthDate'))
    if age is not None:
        if age <= 12:
            age_groups['Child'] += 1
        elif age <= 19:
            age_groups['Teen'] += 1
        elif age <= 59:
            age_groups['Adult'] += 1
        else:
            age_groups['Senior'] += 1

adult_count = age_groups['Adult']

In [20]:
adult_count

21

In [23]:
#7. Calculate the average number of medicines prescribed

#In this we are first calculating the total number of medicines and we are using the total number of patients which we calculated above for female percentage
total_medicines = sum(len(consultation.get('medicines', [])) for consultation in consultation_data)
average_medicines = total_medicines / total_patients
average_medicines = round(average_medicines, 2)

In [24]:
average_medicines

2.13

In [35]:
#8. Determine the 3rd most frequently prescribed medicineName
#In this we count the number of medicines we have for each group and then we use the most_common function to get the third most common medicine
medicine_counter = Counter()
for consultation in consultation_data:
    for medicine in consultation.get('medicines', []):
        medicine_counter[medicine['medicineName']] += 1

third_most_common_medicine = medicine_counter.most_common(3)[2][0]

In [36]:
third_most_common_medicine

'C'

In [38]:
#9. Find the percentage distribution of total active and inactive medicines

#In this we first calculate the sum of total active medicines and the sum of total inactive medicines
total_active = sum(1 for consultation in consultation_data for medicine in consultation.get('medicines', []) if medicine['isActive'])
total_inactive = sum(1 for consultation in consultation_data for medicine in consultation.get('medicines', []) if not medicine['isActive'])
total_medicines = total_active + total_inactive

#Then we calculate the percentage of active and inactive medicines
active_percentage = (total_active / total_medicines) * 100
inactive_percentage = (total_inactive / total_medicines) * 100

# Then finally we round the percentages to 2 decimal places
active_percentage = round(active_percentage, 2)
inactive_percentage = round(inactive_percentage, 2)



In [40]:

df = pd.DataFrame(patient_details)
df.head()

,_id,firstName,lastName,emailId,gender,alternateContact,birthDate,phrId,chat,customId
0,T6hb630b3,Css,,,M,,None,NaN,NaN,NaN
1,T6h33b300,Lokesh,,,M,NaN,1996-05-16T18:30:00.000Z,NaN,NaN,NaN
2,TjhB4373,Shila,Das,,M,NaN,NaN,63b5hvy614d5,NaN,NaN
3,6df4R5b,Bhavika,Ben Panchal,,F,NaN,1988-04-24T14:30:00.000Z,644nig7y,NaN,NaN
4,lK9hy06a,Raghu Viju,,,M,,NaN,NaN,NaN,NaN


In [41]:
# Replace None and empty strings with NaN
df.replace('', pd.NA, inplace=True)
df = df.applymap(lambda x: pd.NA if x is None else x)



In [46]:
# Calculate the percentage of missing values for the specified columns
missing_percentages = df[['firstName', 'lastName', 'birthDate']].isna().mean() * 100

missing_percentages = missing_percentages.round(2)

In [43]:
# Below is the final answers of the above code:

print(f"4. percentage of missing values for firstName, lastName and DOB: {', '.join(missing_percentages.astype(str))}")
print(f"5. Percentage of female gender after imputation: {female_percentage}")
print(f"6. Count of Adults: {adult_count}")
print(f"7. Average number of medicines prescribed: {average_medicines}")
print(f"8. 3rd most frequently prescribed medicineName: {third_most_common_medicine}")
print(f"9. Percentage distribution of total active and inactive medicines: {active_percentage}, {inactive_percentage}")

4. percentage of missing values for firstName, lastName and DOB: 0.0, 70.97, 32.26
5. Percentage of female gender after imputation: 32.26
6. Count of Adults: 21
7. Average number of medicines prescribed: 2.13
8. 3rd most frequently prescribed medicineName: C
9. Percentage distribution of total active and inactive medicines: 69.7, 30.3


In [48]:
def is_valid_phone_number(phone_number):
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]

    if phone_number.isdigit() and 6000000000 <= int(phone_number) <= 9999999999:
        return True
    else:
        return False



In [53]:
# prompt: Check the number of valid phone numbers

valid_phone_numbers = 0
for patient in patient_details:
  if is_valid_phone_number(patient.get('phoneNumber','')):
    valid_phone_numbers += 1
print(f"Number of valid phone numbers: {valid_phone_numbers}")


Number of valid phone numbers: 0


In [55]:
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['numberOfPrescribedMedicines'] = pd.to_numeric(df['numberOfPrescribedMedicines'])

df = df.dropna(subset=['age', 'numberOfPrescribedMedicines'])

pearson_corr = df['age'].corr(df['numberOfPrescribedMedicines'])

pearson_corr = round(pearson_corr, 2)

print(pearson_corr)

KeyError: 'age'